In [ ]:
import ssl
import json
import asyncio
import logging
import websockets
import threading

ws_conns = set() 

async def broadcast_handler(ws_conn):
    global ws_conns 
    try: 
        ws_conns.add(ws_conn)
        # Welcome the new user
        await ws_conn.send(json.dumps({"greeting": "Welcome"})) 
        client_type = ""
        async for msg in ws_conn:
            msg = json.loads(msg)
            if "greeting" in msg: 
                client_type = msg["greeting"] 
                websockets.broadcast(ws_conns, json.dumps({"greeting": f"New {client_type} is online~"}))
                print(f"New {client_type} is online~")
                continue
            elif msg["device_type"] == "virtual_keyboard":  
                val   = msg["value"      ]
                val_t = msg["value_type" ] 
                if val_t == "string":
                    val = ord(val)  
                elif val_t == 'int':
                    val = int(val) 
                else:
                    continue
                print(msg)
                websockets.broadcast(ws_conns, json.dumps(msg)) 
    except websockets.ConnectionClosed:
        pass # This prevents the "connection handler failed" exception
    finally: 
        print(f"Broadcast: A {client_type} is offline")
        websockets.broadcast(ws_conns, json.dumps({"greeting": f"A {client_type} is offline~"}))
        ws_conns.remove(ws_conn) 

async def main(): 
    server_cert = 'fullchain.pem'  # secret of my apache and flask site
    server_key  = 'privkey.pem'    # secret of my apache and flask site  
    ssl_context = ssl.SSLContext(ssl.PROTOCOL_TLS_SERVER)  
    ssl_context.load_cert_chain(certfile=server_cert, keyfile=server_key) 
    bapf = websockets.basic_auth_protocol_factory(
        realm="my dev server",
        credentials=("hello", "iloveyou"),
    )
    async with websockets.serve(broadcast_handler, 
                                host="guoxiaokang.com", 
                                port=8765,
                                create_protocol=bapf,
                                ssl=ssl_context): 
        await asyncio.Future()  # run forever 

def foo(): asyncio.run(main())
threading.Thread(target=foo).start()  